In [1]:
import pickle

In [2]:
fname = '/workspace/data/wikidata_titles_mapping.pickle'

In [3]:
with open(fname, 'rb') as fd:
    mapping = pickle.load(fd)

In [8]:
with open('/workspace/data/wikidata_labels_w_descr_3.pickle', 'rb') as fd:
    mapping_wikidata = pickle.load(fd)

In [6]:
next(iter(mapping.keys()))

6199

In [7]:
mapping[103922126]

KeyError: 103922126

In [9]:
mapping_wikidata[103922126]

['United Kingdom', set(), 'shipwreck off the Scottish coast']

# Twilight bug

In [1]:
import importlib
from ctrie import ConstrainedState, DictIndex, TripleNotFoundException, EmptyIndexException
from transformers import AutoTokenizer
import random

In [3]:
'''
<|im_start|>system
You are a helpful question answering assistant that bases its answer on facts from a knowledge base.
1) You receive an input question.
2) You reason on the path you need to follow to reach the answer starting from the information in the question.
3) You explicitly provide relevant facts, one per line starting with "Fact:".
4) You explain your reasoning process and provide a long answer with your motivations based on the facts.
5) You provide a short concise answer.

Sometimes it could happen that there are no relevant facts in the knowledge base.
In these cases you either:
a) provide your own answer clearly stating that there were no supporting facts to your answer;
b) in case you don't know the answer or you are not confident enough, you just answer "I don't know".
<|im_end|>
<|im_start|>user
Which mountain is taller between Mont Blanc and Mount Rainier?
<|im_end|>
<|im_start|>assistant
Reasoning: I need to provide the height of Mont Blanc and the height of Mount Rainier, then I need to compare the two heights and the final answer will be the taller mountain.
Fact: <Mont Blanc> <elevation above sea level> <4,807.02±0.5 meters> .
Now I need the height of Mount Rainier.
Fact: <Mount Rainier> <elevation above sea level> <4,389 meters> .
Long answer: Mont Blanc is 4,807 meters tall, while Mount Rainier is 4,389 meters, so Mont Blanc is taller than Mount Rainier.

Answer: Mont Blanc.
<|im_end|>
<|im_start|>user
What is the fourth book in the Twilight series?<|im_end|>
<|im_start|>assistant
Reasoning: To answer the question about the fourth book in the Twilight series, I need to recall the titles of the books in the series. However, there are no explicit facts provided about the titles of the books in the Twilight series in the knowledge base. Therefore, I will have to rely on my general knowledge to provide the answer.
Fact: <Twilight> <author> <Julia Frankau> .
Fact: <Twilight> <author> <Paul Laurence Dunbar> .
Fact: <Twilight> <genre> <Science fiction> .
Fact: <Twilight> <genre> <Soul music> .
Fact: <Twilight> <genre> <Rock music> .
Fact: <Twilight> <genre> <Pop music> .
Fact: <Twilight> <genre> <Alternative rock> .
Fact: <Twilight> <genre> <Jazz> .
Fact: <Twilight> <genre> <Techno> .
Fact: <Twilight> <genre> <Experimental film> .
Fact: <Twilight> <genre> <Video art> .
Fact: <Twilight> <genre> <Landscape painting> .
Fact: <Twilight> <genre> <Public art> .
Fact: <Twilight> <genre> <Silent film> .
Fact: <Twilight> <genre>
Fact: <Twilight> <genre>
Fact: <Twilight> <genre>
Fact: <Twilight> <genre>
Fact: <Twilight> <genre>
Fact: <Twilight> <genre>
Fact: <Twilight> <genre>
Fact: <Twilight> <genre>
Fact: <Twilight> <genre>
Fact: <Twilight> <genre>
Fact: <Twilight> <genre>
Fact: <Tw'''

'''
Fact: <Twilight> <author> <Fitz-Greene Halleck> .
Fact: <Twilight> <author> <Eduard von Keyserling> .
Fact: <Twilight> <author>
Fact: <Twilight> <author>
Fact: <Twilight> <author>
'''

'\nFact: <Twilight> <author> <Fitz-Greene Halleck> .\nFact: <Twilight> <author> <Eduard von Keyserling> .\nFact: <Twilight> <author>\nFact: <Twilight> <author>\nFact: <Twilight> <author>\n'

In [4]:
index_module = importlib.import_module('qwen25_index')
Index = getattr(index_module, 'Index')
index = Index()

In [5]:
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen2.5-3B-Instruct')

In [68]:
state = ConstrainedState(
                    begin_pattern = [17417, 25],
                    end_pattern = 198,
                    cache_index = DictIndex(end_of_triple=index.end_of_triple),
                    subtree_cache = DictIndex(end_of_triple=index.end_of_triple),
                    oneleaf_cache = DictIndex(end_of_triple=index.end_of_triple),
                )

In [69]:
state.state = state.CONSTRAINED_GENERATION
state.cursor = 6

In [70]:
tokenizer.tokenize('''
Fact: <Twilight> <author''')[3:]

['Ġ<', 'Tw', 'ilight', '>', 'Ġ<', 'author']

In [71]:
input_sequence = tokenizer.encode('''
Fact: <Twilight> <author''')[3:]
print(input_sequence)

[366, 22816, 34900, 29, 366, 3094]


In [72]:
random.seed(1234)

In [73]:
#input_sequence = [366, 22816, 34900, 29, 366, 33613]
#input_sequence = [366, 22816, 34900, 29, 366, 33613, 29, 366, 28256, 78, 29, 659]

In [74]:
input_sequence

[366, 22816, 34900, 29, 366, 3094]

In [86]:
tokenizer.decode(29)

'>'

In [75]:
input_sequence_run = input_sequence.copy() #[366, 22816, 34900, 29, 366, 33613]
for i in range(100):
    possible_tokens = index.index.next_tokens(input_sequence_run, state = state)
    try:
        visited_tokens = state.cache_index.next_tokens(input_sequence_run)
        # print(visited_tokens, end=' = ')
        index.index.subtract_tokens(possible_tokens, visited_tokens)
        #print('sub')
    except EmptyIndexException:
        # ignore when the cache index is empty
        pass
    except TripleNotFoundException:
        # ignore if triple not in cache index
        pass

    if len(possible_tokens) == 1 and 659 in possible_tokens:
        # end
        input_sequence_run = input_sequence_run + [659]
        state.cache_index.add(input_sequence_run)
        print(tokenizer.decode(input_sequence_run))
        #input_sequence = []
        input_sequence_run = input_sequence.copy() #[366, 22816, 34900, 29, 366, 33613]

    else:
        new_token = random.choice(list(possible_tokens.keys()))
        input_sequence_run.append(new_token)

        #print(input_sequence)


 <Twilight> <author> <Fitz-Greene Halleck> .
 <Twilight> <author> <Howard Chaykin> .


IndexError: Cannot choose from an empty sequence

In [85]:
index.index.next_tokens([366, 22816, 34900, 29, 366, 3094, 29], state = state)

{366: 2}

In [77]:
tokenizer.decode(input_sequence_run)


' <Twilight> <author>'

In [80]:
input_sequence_run

[366, 22816, 34900, 29, 366, 3094, 29]

In [36]:
print(tokenizer.decode(input_sequence))
print(possible_tokens, tokenizer.convert_ids_to_tokens(possible_tokens.keys()))
try:
    print(visited_tokens)
except NameError:
    pass

 <Twilight> <genre>
{} []
{366: 12}


In [ ]:
input_sequence

In [37]:
state.is_constrained()

True

In [38]:
state.cache_index

[0, []]

In [87]:
'''
postgres=# select id, key, numleaves, children, childrenleaves from ctriev3 where key = array[150000, 366, 23662, 36000, 29, 366,3170];
                                                                             
    id     |                 key                  | numleaves |                                                    children                                                     | childrenleaves 
-----------+--------------------------------------+-----------+-----------------------------------------------------------------------------------------------------------------+----------------
  39754563 | {150000,366,23662,36000,29,366,3170} |         1 | {29,366,37659,7957,320,18688,16401,662}                                                                         | {}
  48017622 | {150000,366,23662,36000,29,366,3170} |         1 | {29,366,29185,689,9454,2933,29,662}                                                                             | {}
  59855946 | {150000,366,23662,36000,29,366,3170} |         1 | {29,366,23083,432,13,10058,14767,29,662}                                                                        | {}
 104236583 | {150000,366,23662,36000,29,366,3170} |         1 | {29,366,56,40055,71841,73,56761,320,52566,40360,11,69435,380,323,32465,18640,320,9378,17,12,4468,15,595,29,662} | {}
 109789191 | {150000,366,23662,36000,29,366,3170} |         2 | {29}                                                                                                            | {2}
 161521985 | {150000,366,23662,36000,29,366,3170} |         1 | {29,366,45887,21334,320,3170,16401,662}                                                                         | {}
 183882233 | {150000,366,23662,36000,29,366,3170} |         1 | {29,366,45887,21334,320,3170,16401,662}                                                                         | {}
 186175949 | {150000,366,23662,36000,29,366,3170} |         1 | {29,366,61760,483,36221,29,662}                                                                                 | {}
 190219190 | {150000,366,23662,36000,29,366,3170} |         1 | {29,366,36,1072,569,6675,5422,805,2785,29,662}                                                                  | {}
 193702832 | {150000,366,23662,36000,29,366,3170} |         1 | {29,366,26368,33767,16848,29838,2308,29,662}                                                                    | {}
 202558649 | {150000,366,23662,36000,29,366,3170} |         1 | {29,366,23083,432,13,10058,14767,29,662}                                                                        | {}
 220277159 | {150000,366,23662,36000,29,366,3170} |         1 | {29,366,76768,921,352,8148,29,662}                                                                              | {}
(12 rows)

postgres=# select id, key, sum(numleaves), children, childrenleaves from ctriev3 where key = array[150000, 366, 23662, 36000, 29, 366,3170];
                                                                             
ERROR:  column "ctriev3.id" must appear in the GROUP BY clause or be used in an aggregate function
LINE 1: select id, key, sum(numleaves), children, childrenleaves fro...
               ^
postgres=# select sum(numleaves)from ctriev3 where key = array[150000, 366, 23662, 36000, 29, 366,3170];
                                                                             
 sum 
-----
  13
(1 row)

postgres=# select count(*), children from ctriev3 where key = array[150000, 366, 23662, 36000, 29, 366,3170] group by children;
                                                                             
 count |                                                    children                                                     
-------+-----------------------------------------------------------------------------------------------------------------
     1 | {29}
     1 | {29,366,36,1072,569,6675,5422,805,2785,29,662}
     1 | {29,366,56,40055,71841,73,56761,320,52566,40360,11,69435,380,323,32465,18640,320,9378,17,12,4468,15,595,29,662}
     2 | {29,366,23083,432,13,10058,14767,29,662}
     1 | {29,366,26368,33767,16848,29838,2308,29,662}
     1 | {29,366,29185,689,9454,2933,29,662}
     1 | {29,366,37659,7957,320,18688,16401,662}
     2 | {29,366,45887,21334,320,3170,16401,662}
     1 | {29,366,61760,483,36221,29,662}
     1 | {29,366,76768,921,352,8148,29,662}
(10 rows)
'''

'\npostgres=# select id, key, numleaves, children, childrenleaves from ctriev3 where key = array[150000, 366, 23662, 36000, 29, 366,3170];\n                                                                             \n    id     |                 key                  | numleaves |                                                    children                                                     | childrenleaves \n-----------+--------------------------------------+-----------+-----------------------------------------------------------------------------------------------------------------+----------------\n  39754563 | {150000,366,23662,36000,29,366,3170} |         1 | {29,366,37659,7957,320,18688,16401,662}                                                                         | {}\n  48017622 | {150000,366,23662,36000,29,366,3170} |         1 | {29,366,29185,689,9454,2933,29,662}                                                                             | {}\n  59855946 | {150000,366,23